In [44]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

# EEG package modules__________________________________________________________________
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}
#2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}
#3 - This is to run the wakeful eyes closed experiments
folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

#Folder = folders[sys.argv[1]] #User can input which experiments will be run
print('Change the folder here ^_^! ')

folder = folders['EC1'] #I'm choosing this because it's the TS & Therefore quickest hopefully?
folder = 'REM_full_expts/'

expts = ['MrSQL_expt_1', 'MrSQL_expt_2' , 'MrSQL_expt_3' , 'MrSQL_expt_4']

power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

regions = ['Prefrontal','Frontal', 'Left Frontal', 'Right Frontal', 'Central', 'Left Central', 
           'Right Central', 'Left Temporal','Right Temporal', 'Parietal', 'Left Parietal', 'Right Parietal', 'Occipital']

#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

Change the folder here ^_^! 


In [47]:
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,1)

In [48]:
X_Delta = X_expt[['Delta_Prefrontal']]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_Delta, y_expt, random_state = 43)

In [86]:
classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) 

In [97]:
%%time
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

CPU times: user 2.12 s, sys: 5.99 ms, total: 2.12 s
Wall time: 2.12 s


0.75

In [98]:
classifier.fit(X_test, y_test)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [22]:
X_Theta = X_expt[['Theta_Prefrontal']]

In [23]:
Xt_train, Xt_test, yt_train, yt_test = train_test_split(X_Theta, y_expt, random_state = 43)

In [26]:
%%time
classifier.fit(Xt_train, yt_train)
yt_pred = classifier.predict(Xt_test)
accuracy_score(yt_test, yt_pred)

CPU times: user 2.14 s, sys: 5.91 ms, total: 2.15 s
Wall time: 2.15 s


5e-01

In [27]:
yt_pred = classifier.predict(Xt_test)
accuracy_score(yt_test, yt_pred)

5e-01

In [35]:
X_Delta

,Delta_Prefrontal
id_val,
0,0 0.451210 1 0.485463 2 0.43161...
1,0 0.556763 1 0.524243 2 0.29737...
2,0 0.518393 1 0.598627 2 0.51664...
3,0 0.579698 1 0.506822 2 0.47951...
4,0 0.637609 1 0.585811 2 0.55187...
5,0 0.620811 1 0.561290 2 0.54175...
6,0 0.629542 1 0.579442 2 0.46030...
7,0 0.622035 1 0.641346 2 0.58693...
8,0 0.649790 1 0.630928 2 0.65337...


## Unravel the loop

In [ ]:
X_Delta,  y_expt , groups_expt, classifier, rand_state = 40

In [49]:
classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

In [53]:
gss = GroupShuffleSplit(n_splits=5, test_size = 0.2,  random_state=40)
acc_scores = []
y_pred_arrays = []
y_true_arrays = []

In [54]:
train_indices = [] 
test_indices = []
for train_index, test_index  in gss.split(X_Delta, y_expt, groups_expt):
    train_indices.append(train_index)
    test_indices.append(test_index)

In [70]:
len(groups_expt)

40

In [68]:
len(X_Delta)

40

In [85]:
np.unique(groups_expt.iloc[train_indices[4]])

array(['1103', '1104', '1105', '1106', '1108', '1109', '1110', '1111',
       '1112', '1113', '1119', '4401', '4402', '4403', '4404', '4405',
       '4406', '4407', '4408', '4409'], dtype=object)

In [83]:
np.unique(groups_expt.iloc[test_indices[4]])

array(['1101', '1102', '1116', '1117', '1118', '4410'], dtype=object)

In [55]:
train_indices

[array([ 0,  1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 13, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 29, 30, 31, 32, 34, 35, 36, 37, 38]),
 array([ 0,  1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 20, 21,
        22, 23, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]),
 array([ 0,  1,  2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 29, 30, 31, 33, 34, 35, 36, 39]),
 array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38])]

In [65]:
test_indices

[array([ 5,  6, 22, 23, 24, 25, 32, 39]),
 array([11, 12, 14, 26, 27, 28, 33, 39]),
 array([ 5,  6, 13, 18, 19, 26, 27, 28, 39]),
 array([ 3,  4, 24, 25, 26, 27, 28, 32, 37, 38]),
 array([ 0,  1,  2, 20, 21, 22, 23, 39])]

In [56]:
X = X_Delta.copy()
y = y_expt.copy()

In [57]:
X_train, X_test = X.iloc[train_indices[0]], X.iloc[test_indices[0]]
y_train, y_test = y.iloc[train_indices[0]], y.iloc[test_indices[0]]

classifier.fit(X_train, y_train)
y_preds = classifier.predict(X_test)

y_pred_arrays.append(y_preds)
y_true_arrays.append(y_test)

acc_scores.append( accuracy_score(y_preds, y_test) )

In [58]:
X_train, X_test = X.iloc[train_indices[1]], X.iloc[test_indices[1]]
y_train, y_test = y.iloc[train_indices[1]], y.iloc[test_indices[1]]

classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

classifier.fit(X_train, y_train)
y_preds = classifier.predict(X_test)

y_pred_arrays.append(y_preds)
y_true_arrays.append(y_test)

acc_scores.append( accuracy_score(y_preds, y_test) )

In [59]:
X_train, X_test = X.iloc[train_indices[2]], X.iloc[test_indices[2]]
y_train, y_test = y.iloc[train_indices[2]], y.iloc[test_indices[2]]

classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

classifier.fit(X_train, y_train)
y_preds = classifier.predict(X_test)

y_pred_arrays.append(y_preds)
y_true_arrays.append(y_test)

acc_scores.append( accuracy_score(y_preds, y_test) )

In [60]:
X_train, X_test = X.iloc[train_indices[3]], X.iloc[test_indices[3]]
y_train, y_test = y.iloc[train_indices[3]], y.iloc[test_indices[3]]

classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

classifier.fit(X_train, y_train)
y_preds = classifier.predict(X_test)

y_pred_arrays.append(y_preds)
y_true_arrays.append(y_test)

acc_scores.append( accuracy_score(y_preds, y_test) )

In [61]:
X_train, X_test = X.iloc[train_indices[4]], X.iloc[test_indices[4]]
y_train, y_test = y.iloc[train_indices[4]], y.iloc[test_indices[4]]

classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

classifier.fit(X_train, y_train)
y_preds = classifier.predict(X_test)

y_pred_arrays.append(y_preds)
y_true_arrays.append(y_test)

acc_scores.append( accuracy_score(y_preds, y_test) )

In [64]:
classifier.fit(X_test, y_test)

accuracy_score( classifier.predict(X_test) , y_test )


1.0

In [62]:
acc_scores = np.array(acc_scores)
mean_acc = acc_scores.mean()
acc_std = acc_scores.std()

In [63]:
print(mean_acc)
print(acc_std)

0.5661111111111111
0.16239336482218603


## Group Shuffle Split CV Function for MrSEQL

In [73]:
def run_clf_with_CV(X, y , groups , grant_clf , rand_state = 40):
    '''
    Run classifier on input features X -> seperate more complex version of this function required for column ensemble models
    Cross validation is performed
    Output is : Mean of CV accuracy values and Std of CV accuracy values, y_pred arrays for each of 5 folds , y_true arrays for each of the 5 folds
    '''
    #The test function
    gss = GroupShuffleSplit(n_splits=5, test_size = 0.2,  random_state=rand_state)
    acc_scores = []
    y_pred_arrays = []
    y_true_arrays = []

    for train_index, test_index  in gss.split(X, y, groups):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


                    grant_clf.fit(X_train, y_train)
                    y_preds = grant_clf.predict(X_test)
                        
                    y_pred_arrays.append(y_preds)
                    y_true_arrays.append(y_test)

                    acc_scores.append( accuracy_score(y_preds, y_test) )
    
    acc_scores = np.array(acc_scores)
    mean_acc = acc_scores.mean()
    acc_std = acc_scores.std()

   
    return mean_acc, acc_std , y_pred_arrays , y_true_arrays 

In [49]:
def run_clf_with_CV_init_inside(X, y , groups , clf , rand_state = 40):
    '''
    Run classifier on input features X -> seperate more complex version of this function required for column ensemble models
    Cross validation is performed
    Output is : Mean of CV accuracy values and Std of CV accuracy values, y_pred arrays for each of 5 folds , y_true arrays for each of the 5 folds
    '''
    #The test function
    gss = GroupShuffleSplit(n_splits=5, test_size = 0.2,  random_state=rand_state)
    acc_scores = []
    y_pred_arrays = []
    y_true_arrays = []

    for train_index, test_index  in gss.split(X, y, groups):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
                    clf = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])


                    clf.fit(X_train, y_train)
                    y_preds = clf.predict(X_test)
                        
                    y_pred_arrays.append(y_preds)
                    y_true_arrays.append(y_test)

                    acc_scores.append( accuracy_score(y_preds, y_test) )
    
    acc_scores = np.array(acc_scores)
    mean_acc = acc_scores.mean()
    acc_std = acc_scores.std()

   
    return mean_acc, acc_std , y_pred_arrays , y_true_arrays 

In [74]:
classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax'])

In [75]:
mean_acc, acc_std , y_pred_arrays , y_true_arrays  = run_clf_with_CV(X_Delta,  y_expt , groups_expt, classifier, rand_state = 40 ) 

In [76]:
mean_acc

0.8577777777777778

In [77]:
acc_std

0.054698951622878245

In [50]:
mean_acc, acc_std , y_pred_arrays , y_true_arrays  = run_clf_with_CV_init_inside(X_Delta,  y_expt , groups_expt, classifier, rand_state = 40 ) 

In [53]:
acc_std

0.16239336482218603

In [52]:
mean_acc

0.5661111111111111

## Group Shuffle Split CV Function for other classifier

In [65]:
def run_clf_with_CV_init_inside_KNN(X, y , groups , clf , rand_state = 40):
    '''
    Run classifier on input features X -> seperate more complex version of this function required for column ensemble models
    Cross validation is performed
    Output is : Mean of CV accuracy values and Std of CV accuracy values, y_pred arrays for each of 5 folds , y_true arrays for each of the 5 folds
    '''
    #The test function
    gss = GroupShuffleSplit(n_splits=5, test_size = 0.2,  random_state=rand_state)
    acc_scores = []
    y_pred_arrays = []
    y_true_arrays = []

    for train_index, test_index  in gss.split(X, y, groups):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
                    clf =  KNeighborsTimeSeriesClassifier()


                    clf.fit(X_train, y_train)
                    y_preds = clf.predict(X_test)
                        
                    y_pred_arrays.append(y_preds)
                    y_true_arrays.append(y_test)

                    acc_scores.append( accuracy_score(y_preds, y_test) )
    
    acc_scores = np.array(acc_scores)
    mean_acc = acc_scores.mean()
    acc_std = acc_scores.std()

   
    return mean_acc, acc_std , y_pred_arrays , y_true_arrays 

In [58]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [62]:
clf =  KNeighborsTimeSeriesClassifier()

In [63]:
mean_acc, acc_std , y_pred_arrays , y_true_arrays  = run_clf_with_CV(X_Delta,  y_expt , groups_expt, clf, rand_state = 40 ) 

In [64]:
print(mean_acc)
print(acc_std)

0.6405555555555555
0.19506725652425944


In [66]:
mean_acc, acc_std , y_pred_arrays , y_true_arrays  = run_clf_with_CV_init_inside_KNN(X_Delta,  y_expt , groups_expt, clf, rand_state = 40 ) 

In [67]:
print(mean_acc)
print(acc_std)

0.6405555555555555
0.19506725652425944
